In [1]:
import pandas as pd
import pandas as pd
import re
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors

# Dataset containing links to the reports
Downloaded file from the Sustainability Reporting Navigator

https://docs.google.com/spreadsheets/d/1Nlyf8Yz_9Fst8rEmQc2IMc-DWLF1fpmBTB7n4FlZwxs/edit?gid=0#gid=0 
date: 14.08.2025

In [2]:
# read in csv
srn_reports_raw = pd.read_csv("SRN-CSRD_report_archive - csrd.csv", header=2)
print(srn_reports_raw.head())

  verified      srn_id        company          isin  country publication date  \
0      yes  tVkXa1OoYn     Netcompany  DK0060952919  Denmark       2025-01-29   
1      yes  4P9CZx30gn           Tryg  DK0060636678  Denmark       2025-01-23   
2      yes  u63Ki9uBLF  DSV Panalpina  DK0060079531  Denmark       2025-02-04   
3      yes  PtoavXijSQ       Lundbeck  DK0061804697  Denmark       2025-02-04   
4      yes  g5GXXjndDy         Vestas  DK0061539921  Denmark       2025-02-05   

  claim full CSRD compliance   auditor  start PDF  end PDF  pages PDF  \
0                       True        EY       62.0    154.0       93.0   
1                       True       PwC       53.0    131.0       79.0   
2                       True       PwC       42.0     84.0       43.0   
3                       True       PwC       59.0    144.0       86.0   
4                       True  Deloitte       51.0    132.0       82.0   

                                                link ESEF-\ntagged  \
0   

In [3]:
# drop all rows where 'verified' is not yes
srn_reports = srn_reports_raw[srn_reports_raw['verified'] == 'yes']
print(len(srn_reports))

# keep only relevant columns
srn_reports = srn_reports[['company', 'isin', 'country', 'publication date', 'auditor', 'link', 'SASB industry \n(SICS® Industries)']]
# rename columns
srn_reports = srn_reports.rename(columns={
    'publication date': 'publication_date',
    'SASB industry \n(SICS® Industries)': 'SASB_industry'
})

627


In [4]:
# drop all where isin "n.a" or "n.a."
srn_reports = srn_reports[~srn_reports['isin'].isin(['n.a', 'n.a.'])]

print(len(srn_reports))

576


In [5]:
# new column for report_id (company name + year)
# preparing filenames
def prepare_filename(name):
    return re.sub(r'[\\/*?:"<>|]', "", name)

for index, row in srn_reports.iterrows():
    company_name = row['company']
    report_id = f"{prepare_filename(company_name)}_2024".replace(" ", "")
    srn_reports.at[index, 'report_id'] = report_id


In [6]:
# save as excel
srn_reports.to_excel("srn_reports.xlsx", index=False)

# Dataset with company info

retrieved all company data with the help of ISIN code from LSEG (where isin was available)

In [ ]:
company_data = pd.read_excel("company_data.xlsx")

In [ ]:
# drop NAs
company_data = company_data.dropna()
print(len(company_data))

506


In [12]:
# drop all companys with above 750 employees
company_data = company_data[company_data['Number of Employees'] >= 750]
print(len(company_data))

485


In [ ]:
# keep only srn information on companies that are in company_data
esrs_reports = srn_reports[srn_reports['isin'].isin(company_data['ISIN'])]
print(len(esrs_reports))

482


In [19]:
# save as excel
esrs_reports.to_excel("esrs_reports.xlsx", index=False)